# Fixed number of particles and symbolic results 

In many situations, we are interested in fermionic states with a fixed number of particles. This could happen, for instance, when dealing with the eigenstates of the most common Hamiltonians. Given that situation, we can reduce the basis of our system by a lot. More exactly, if d is the dimension and n the number of particles, we can go from $2^d\times 2^d$ to $\binom{d}{n}\times \binom{d}{n}$ systems. This can be very useful for diagonalizing Hamiltonians, or many other situations.

It is also of interest to work with non-defined variables in order to look for analytical results. This can be done integrating the Fermionic package together with SymPy via PyCall. There are also packages for defining symbolic fermionic operators. In this notebook we will see how exactly does this work.

1. [Operators with fixed number](#opfix)
2. [States with fixed number](#statefix)
3. [Symbolic interaction](#symbol)
4. [Symbolic operators](#sops)


<a id="opfix"></a>
## Operators with fixed number



In [1]:
using Fermionic

┌ Info: Precompiling Fermionic [9854c450-6826-470b-b7cd-d3803b32f4ba]
└ @ Base loading.jl:1260


We will use a function called *fixed()* which takes 2 arguments: the operator we want to reduce and the number of particles we want to fix to. For operators that preserve particle number, this will result in the elimination of empty rows and colums, allowing us to do matrix operations in smaller dimensions.

In [3]:
o = Op(4)
cdcm(o,1,2) 

16×16 SparseArrays.SparseMatrixCSC{Float64,Int64} with 4 stored entries:
  [9 , 5]  =  1.0
  [10, 6]  =  1.0
  [11, 7]  =  1.0
  [12, 8]  =  1.0

Notice we are using a number-preserving operation. We will now fix this operation for 2 particles states.

In [4]:
fixed(cdcm(o,1,2), 2) 

6×6 SparseArrays.SparseMatrixCSC{Float64,Int64} with 2 stored entries:
  [4, 2]  =  1.0
  [5, 3]  =  1.0

Notice the dimension has dropped from 16x16 to 6x6 (in general, from $2^d\times 2^d$ to $\binom{d}{n}\times \binom{d}{n}$). This new state is written in a new reduced basis. We can always access the new basis, with fixed number parity with the function *basis_m()* which also takes two arguments: the operators Op() and the number of particles.

In [6]:
b = basis_m(o,2);
Matrix(b)

6×4 Array{Float64,2}:
 0.0  0.0  1.0  1.0
 0.0  1.0  0.0  1.0
 0.0  1.0  1.0  0.0
 1.0  0.0  0.0  1.0
 1.0  0.0  1.0  0.0
 1.0  1.0  0.0  0.0

So the operator $c_1^\dagger c_2$ connects these two states

In [16]:
println(Matrix(b)[4,:])
println(Matrix(b)[2,:])
println("--------------------")
println(Matrix(b)[5,:])
println(Matrix(b)[3,:])

[1.0, 0.0, 0.0, 1.0]
[0.0, 1.0, 0.0, 1.0]
--------------------
[1.0, 0.0, 1.0, 0.0]
[0.0, 1.0, 1.0, 0.0]


We can do this for every operator, but it only really makes sense for those that are number preseving.
Some really nice examples are the superconducting and the Lipkin hamiltonians, which are also solved in the /examples folder.

<a id="statefix"></a>
## States with fixed number

We already shown how to work with states in order to access some properties, such as the one body matrix and its corresponding entropy. If we are working with fixed number, we can do the same thing. The new types are:

- State_fixed: for array states with real coefficients and with fixed number
- State_complex_fixed: for array states with complex coefficients and with fixed number
- State_sparse_fixed: for sparse states with real coefficients and with fixed number
- State_sparse_complex_fixed: for sparse states with complex coefficients and with fixed number

These states are written in the basis we obtained with basis_m() and should be normalized for proper results. The arguments for these types are the array/sparse array, the operators and the number of fixed particles.




In [2]:
stat = zeros(binomial(4,2)); 
stat[1] = 1;
stat[6] = 1;
stat = stat/sqrt(stat'*stat)

6-element Array{Float64,1}:
 0.7071067811865475
 0.0
 0.0
 0.0
 0.0
 0.7071067811865475

In [4]:
state = State_fixed(stat,o,2);
rhosp(state)

4×4 Array{Float64,2}:
 0.5  0.0  0.0  0.0
 0.0  0.5  0.0  0.0
 0.0  0.0  0.5  0.0
 0.0  0.0  0.0  0.5

In [5]:
eigensp(state)

4-element Array{Float64,1}:
 0.5
 0.5
 0.5
 0.5

In [6]:
ssp(state)

1.0

<a id="symbol"></a>
## Symbolic Algebra

In this section we will learn to interact with the symbolic tools defined in [SymPy](https://github.com/JuliaPy/SymPy.jl). This library works perfectly within Julia via PyCall. I will show you how:

First of all, if you have not done ir yet, you should install Conda:

`using Pkg
Pkg.add("Conda") 
using Conda
Conda.update()
Pkg.add("PyCall")`

We then call PyCall

In [8]:
using PyCall
sympy = pyimport("sympy");

With this amazing package we can work with symbols as they were variables

In [9]:
x = sympy.Symbol("x")      # PyObject x

PyObject x

In [10]:
sympy.Matrix([x, 2, 3])

PyObject Matrix([
[x],
[2],
[3]])

So, for instance, we can exactly diagonalize the superconducting hamiltonian in a symbolic way:
$H=\sum_k \epsilon_k (c_k^\dagger c_k + c_{\bar{k}}^\dagger c_{\bar{k}})-\sum_{k,k'} G_{k,k'} c_{k'}^\dagger c_{\bar{k'}}^\dagger c_{\bar{k}} c_k$
We will set $\epsilon_k = \epsilon \; \forall k$, and $G_{kk'} = g \; \forall k,k'$.

In [11]:
using Fermionic
using SparseArrays

g = sympy.Symbol("g")
e = sympy.Symbol("e")

o = Op(4); #this is for dimension 4
e0 = 1.0 #the energy spectrum of the free term
d = dim(o) 
epsilon = [e0*(i-d/4-1/2) for i in 1:d/2]
epsilon = sort([epsilon; epsilon])

h0 = sum([epsilon[i]*(cdm(o,i)*cm(o,i) + cdm(o,i+1)*cm(o,i+1)) for i in 1:2:(Int(d)-1)])

hi = sum([sum([if i==j spzeros(2^d, 2^d) else (cdm(o,j)*cdm(o,j+1)*cm(o,i+1)*cm(o,i)) end
                    for i in 1:2:(Int(d)-1)]) for j in 1:2:(Int(d)-1)]);
    
mh0 =  Matrix(h0);
mhi = Matrix(hi)

mh = e*sympy.Matrix(mh0) - g*sympy.Matrix(mhi)
(P, D) = mh.diagonalize();
D #this outputs the diagonal matrix.

PyObject Matrix([
[0, 0, 0, 0, 0, 0,      0,      0,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0, 0,      0,      0,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0, 0,      0,      0,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0, 0,      0,      0,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0, 0,      0,      0,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0, 0,      0,      0,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0, 0, -0.5*e,      0,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0, 0,      0, -0.5*e,      0,      0,     0,     0,     0,     0,                   0,                  0],
[0, 0, 0, 0, 0

The eigenvalues are the columns of the following matrix:

In [12]:
P

PyObject Matrix([
[1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,                          0,                          0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0,                          0,                          0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0,   0,                          0,                          0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, g/(e + (e**2 + g**2)**0.5), g/(e - (e**2 + g**2)**0.5)],
[  0,   0,   0,   0,   0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,                          0,                          0],
[  0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,                          0,                          0],
[  0,   0, 1.0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,                          0,                          0],
[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0, 1.0,   0,                

You can learn more about matrix manipulation in [this tutorial](https://docs.sympy.org/latest/modules/matrices/matrices.html).

<a id="sops"></a>
## Symbolic second quantization

There is a library inside SymPy which allows the definition of fermionic symbolic operators. You can finde the complete list of functions in [this link](https://docs.sympy.org/latest/modules/physics/secondquant.html).

It is good enough for working with commutators, but for some reasons the anticommutator identity is not defined for fermions, making it a little limited for complete usage.

In [114]:
sp = pyimport("sympy");
sq = pyimport("sympy.physics.secondquant");
adi = sq.Dagger(sq.F(sp.Symbol("i")));
adj = sq.Dagger(sq.F(sp.Symbol("j")))

PyObject CreateFermion(j)

In [115]:
ak = sq.Dagger(sq.Fd(sp.Symbol("k")))

PyObject AnnihilateFermion(k)

We can evauate contractions:

In [64]:
sq.contraction(adi,adi) #double creation operator

PyObject 0

In [122]:
sq.contraction(ak,ak) #double destruction operator

PyObject 0

In [123]:
a = sq.contraction(adi,ak)

PyObject KroneckerDelta(_i, k)*KroneckerDelta(i, k)

In [125]:
sq.evaluate_deltas(a)

PyObject KroneckerDelta(_i, i)

We can define commutators

In [116]:
com = sq.Commutator(adi*adj,ak)

PyObject Commutator(CreateFermion(i)*CreateFermion(j),f(k))

and expand them with doit()

In [119]:
com.doit(wicks = true) #wicked defines a normal ordering of terms (see Wick's expantion)

PyObject -KroneckerDelta(_a, i)*KroneckerDelta(i, k)*CreateFermion(j) + KroneckerDelta(_a, j)*KroneckerDelta(j, k)*CreateFermion(i) - KroneckerDelta(_i, k)*KroneckerDelta(i, k)*CreateFermion(j) + KroneckerDelta(_i, k)*KroneckerDelta(j, k)*CreateFermion(i)

Also perform a wick expantion

In [67]:
sq.wicks(adi*(ak+adj))

PyObject KroneckerDelta(_i, k)*KroneckerDelta(i, k) + NO(CreateFermion(i)*AnnihilateFermion(k)) + NO(CreateFermion(i)*CreateFermion(j))

In [68]:
sq.wicks(adi*(ak+adj), keep_only_fully_contracted = true)

PyObject KroneckerDelta(_i, k)*KroneckerDelta(i, k)

NO() performs a normal order, assuming anticommutation relations

In [96]:
sq.NO(ak*adi)

PyObject -NO(CreateFermion(i)*AnnihilateFermion(k))

In [126]:
adj = sq.Dagger(sq.CreateFermion(sp.Symbol("j")))

PyObject AnnihilateFermion(j)

Each state can only have one particle, so we choose to store a list of occupied orbits rather than a tuple with occupation numbers (zeros and ones).

In [139]:
f1 = sq.FKet([1,2])

PyObject FockStateKet((1, 2))

In [140]:
f2 = sq.FBra([1,3])

PyObject FockStateKet((1, 3))

In [141]:
sq.InnerProduct(f2,f1)

PyObject 0